# Location selection demo

In [8]:
%matplotlib qt

import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

plt.ion()

n = 20


## Helper functions

In [73]:

def generate_cities(n, seed):
    random.seed(seed)
    # Generate n cities with random coordinates
    cities = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    return cities

def calculate_distance(city1, city2):
    # Calculate Euclidean distance between two cities
    return np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

def create_popuations(n):
    populations = np.array([(random.triangular(-1, 1)**2)*1000 for _ in range(n)]).round() * 100
    return populations


def create_adjacency_matrix(cities):
    n = len(cities)
    adjacency_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                adjacency_matrix[i][j] = calculate_distance(cities[i], cities[j])
    
    return adjacency_matrix

def plot_cities(cities, marker_downscale_factor):
    # Plot the cities on a Cartesian coordinate system
    city_coords = cities.loc[:,["lat", "long"]].values
    x_coords, y_coords = zip(*city_coords)

    plt.figure(figsize=(8, 8))
    plt.scatter(x_coords, y_coords, color='blue', s=cities.loc[:,"pop"]/marker_downscale_factor)
    
    for idx, city in cities.iterrows():
        plt.annotate(city["city"], (city["lat"], city["long"]), textcoords="offset points", xytext=(0,10), ha='center')


    plt.title('Cities on Cartesian Coordinate System')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.xlim(min(x_coords), max(x_coords))
    plt.ylim(min(y_coords), max(y_coords))
    plt.grid()
    plt.show()


## Setting up data

In [74]:

cities = pd.read_csv("cities.csv")

city_coords = cities.loc[:, ["lat","long"]].values

adjacency_matrix = create_adjacency_matrix(city_coords)

populations =  cities.loc[:,"pop"].values

print("Cities (x, y coordinates):")
for index, city in cities.iterrows():
    city_name = city["city"]
    lat  = city["lat"]
    long  = city["long"]
    print(f"{city_name}, {lat}, {long} ")

display(populations)

marker_upscale_factor = 100

plot_cities(cities, marker_downscale_factor)
#plot_cities(cities)


Cities (x, y coordinates):
Ada, 40.768056, -83.825278 
Lima, 40.746389, -84.123333 
Kenton, 40.646667, -83.6225 
Delphos, 40.861111, -84.35 
Bluffton, 40.889444, -83.879167 
Spencerville, 41.099722, -82.125556 
Elida, 40.786667, -84.198889 
Forest, 40.805, -83.511667 
Alger, 40.709722, -83.844167 
Dunkirk, 40.788056, -83.642778 
MountVictory, 40.533611, -83.521111 
Cairo, 40.830833, -84.084444 
St.Marys, 40.548889, -84.373333 


array([ 5334, 35579,  7947,  7117,  3967,  2198,  1923,  1350,   837,
         774,   601,   517,  8397])

## Representation of problem

In [4]:
from pymoo.core.problem import Problem
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga3 import NSGA3

class clinicOptimization(Problem):

    def __init__(self, cities, max_city, city_pops, min_atp, home):
        city_n = len(cities)
        super().__init__(n_var=city_n, n_obj=3, n_ieq_constr=1, xl=0, xu=1, vtype=bool)
        self.max_city = max_city
        self.city_pops = city_pops
        self.min_atp = min_atp
        self.home = home


    def _evaluate(self, x, out, *args, **kwargs):
        x_bool = x < 0.5
        np.multiply(x_bool, self.city_pops)

        visited_per_solution = np.multiply(x_bool, self.city_pops)

        # Objective 1: Maximize total population visited 
        total_pop_visited = visited_per_solution.sum(axis=1)

        # Objective 2: Maximize the total number of cities visited 
        total_cities_visited = x_bool.sum(axis=1)

        # Objective 3: Minimize the total number of events that are overstaffed
        overstaffed_events = np.logical_and(visited_per_solution < self.min_atp, visited_per_solution != 0)
        total_overstaffed_events = overstaffed_events.sum(axis=1)

        # Wrap up our objectives all nice and pretty
        out["F"] = np.column_stack([
            -total_pop_visited,              # max
            -total_cities_visited,           # max
            total_overstaffed_events])       # min

        # Constrains: Total cities visited
        out["G"] = total_cities_visited < self.max_city       




# atp = attendees per site 
# (this is just going off of raw pop, though normally we'd want to estimate the total 
# number of visitors per capital)
min_atp = 1000

home = np.array([50,50])
prob = clinicOptimization(cities, 10, populations, min_atp, home)

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)

algorithm = NSGA3(pop_size=200,
                  ref_dirs=ref_dirs)

res = minimize(prob,
               algorithm,
               ('n_gen', 100),
               verbose=False)

display(res.X < 0.5)
display(res.F)





array([[ True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True, False,  True,  True,  True, False,  True,  True,
         True, False],
       [ True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True, False,  True,  True,  True, False,  True, False,
         True, False],
       [ True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True,  True,  True,  True,  True, False,  True,  True,
         True, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ T

array([[-1.866e+05, -1.700e+01,  3.000e+00],
       [-1.852e+05, -1.500e+01,  1.000e+00],
       [-1.843e+05, -1.400e+01,  0.000e+00],
       [-1.874e+05, -1.900e+01,  5.000e+00],
       [-1.861e+05, -1.600e+01,  2.000e+00],
       [-1.875e+05, -2.000e+01,  6.000e+00],
       [-1.870e+05, -1.800e+01,  4.000e+00]])

In [5]:
import pandas as pd

results = pd.DataFrame(res.F, 
                       columns=["Total Population Visited", "Total Cities visited", "Number of over staffed events"])
results.loc[:,"Total Population Visited"] = np.abs(results.loc[:,"Total Population Visited"])
results.loc[:,"Total Cities visited"] = np.abs(results.loc[:,"Total Cities visited"])

results

,Total Population Visited,Total Cities visited,Number of over staffed events
0,186600.0,17.0,3.0
1,185200.0,15.0,1.0
2,184300.0,14.0,0.0
3,187400.0,19.0,5.0
4,186100.0,16.0,2.0
5,187500.0,20.0,6.0
6,187000.0,18.0,4.0


In [6]:
from mpl_toolkits.mplot3d import Axes3D

x = results.loc[:,"Total Population Visited"]
y = results.loc[:,"Total Cities visited"]
z = results.loc[:,"Number of over staffed events"]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z,
           linewidths=1, alpha=.7,
           edgecolor='k',
           s = 200,
           c=z)

ax.set_xlabel("Total population visited")
ax.set_ylabel("Total cities visited")
ax.set_zlabel("Number of overstaffed events")

plt.show()

In [7]:
#import numpy as np
#from mpl_toolkits.mplot3d import Axes3D
#
## Generate random data for the scatter plot
#num_points = 100
#x = results.loc[:,"Total Population Visited"]
#y = results.loc[:,"Total Cities visited"]
#z = results.loc[:,"Number of over staffed events"]
#
## Create a 2x2 figure
#fig = plt.figure(figsize=(10, 10))
#
## Define the viewing angles for each subplot
#view_angles = [(30, 30), (30, -30), (30, -60), (30, -180)]
#
## Create subplots
#for i in range(2):
#    for j in range(2):
#        ax = fig.add_subplot(2, 2, i * 2 + j + 1, projection='3d')
#        ax.scatter(x, y, z, c = z)
#        ax.set_title(f'View angle: {view_angles[i * 2 + j]}')
#        ax.view_init(elev=view_angles[i * 2 + j][0], azim=view_angles[i * 2 + j][1])
#
## Adjust layout
#plt.tight_layout()
#plt.show()